In [2]:
import pandas as pd
import numpy as np
import re
#import contractions
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import gensim.downloader as api
import gensim.models
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.autograd import Variable

In [3]:
!python --version

Python 3.12.3


In [4]:
torch.__version__

'2.6.0+cu124'

##### VERSIONS OF OTHER LIBRARIES USED

scikit-image : 0.19.2 <br>
numpy : 1.21.5 <br>
pandas : 1.4.2 <br>
contractions : 0.1.73 <br>
gensim : 4.1.2 <br>

# Dataset Generation

In [4]:
#download data: https://www.kaggle.com/datasets/magodobolo/amazon-dataset-reviews-beauty
import os
os.chdir('/home/faculty10/tuanle/Teaching/s25-nlp/week6/')
df = pd.read_csv('amazon_reviews_beauty.csv', header=0, sep=',', quotechar='"', on_bad_lines='skip', dtype='unicode')

In [5]:
df

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True
...,...,...,...,...,...,...,...,...,...,...
701523,4.0,Four Stars,Conditioner is great shampoo not as I expected,[],B006YUIWKA,B006YUIWKA,AFIXGFVEGLMOTMBTJL7H3VSIETDQ,1478227021000,0,True
701524,1.0,Pretty,Did not work! Used the whole bottle and my hai...,[],B006YUIWKA,B006YUIWKA,AFV7YZFOJF564EZGET5LG45K4QEA,1480908730000,0,False
701525,5.0,Great sunless tanner,Product as expected. Shipping was on time.,[],B06ZZV9MZT,B06ZZV9MZT,AHYDCWDMMVMLBX7FY7M7JKADKRDQ,1590547974067,0,True
701526,5.0,The Crown on top is a Ring!!!,"Not only is it a delicious fragrance, but also...",[],B000HB6VLE,B000HB6VLE,AF6ZIAEN7TQ2WY5ZL77F6JDPV7XQ,1184798209000,4,False


In [6]:
df = pd.concat([df.iloc[:, 0], df.iloc[:, 2]], axis = 1)
df

,rating,text
0,5.0,This spray is really nice. It smells really go...
1,4.0,"This product does what I need it to do, I just..."
2,5.0,"Smells good, feels great!"
3,1.0,Felt synthetic
4,5.0,Love it
...,...,...
701523,4.0,Conditioner is great shampoo not as I expected
701524,1.0,Did not work! Used the whole bottle and my hai...
701525,5.0,Product as expected. Shipping was on time.
701526,5.0,"Not only is it a delicious fragrance, but also..."


In [7]:
df = df.dropna(subset=['rating','text'])

In [8]:
pd.options.mode.chained_assignment = None 
def labelClass(rating):
    if rating == "1.0" or rating == "2.0":
          return 1
    if rating == "3.0" :
          return 2
    if rating == "4.0" or rating  == "5.0":
          return 3
df['class'] = df['rating'].map(labelClass)

In [9]:
# CREATE A BALANCED DATASET OF 60K REVIEWS, LIKE HW1
class1 = df.loc[df['class'] == 1].sample(n=20000, random_state=1)
class2 = df.loc[df['class'] == 2].sample(n=20000, random_state=1)
class3 = df.loc[df['class'] == 3].sample(n=20000, random_state=1)
df = pd.concat([class1, class2, class3])

In [10]:
df['text'] = df['text'].str.lower()  # CONVERT ALL REVEIWS TO LOWERCASE
df['text'] = df['text'].astype(str)  
df['text'] = df['text'].apply(lambda x: re.sub(re.compile('http\S+|https\S+'), "", x))  # REMOVE ALL URLs 
df['text'] = df['text'].apply(lambda x: re.sub(re.compile('<.*?>'), "", x))   # REMOVE ALL HTML TAGS
df['text'] = df['text'].apply(lambda x: re.sub(re.compile("[^A-Za-z]")," ", x))  # REMOVE ALL NON-ALPHABETICAL CHARACTERS
df['text'] = df['text'].apply(lambda x: re.sub(re.compile(' +'),' ', x))  # REMOVES EXTRA SPACES IN REVIEWS
df['text'] = df['text'].apply(lambda x: contractions.fix(x))  # PERFORM CONTRACTIONS ON REVIEWS

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipykernel_37037/1946699171.py:3: SyntaxWarning: invalid escape sequence '\S'
  df['text'] = df['text'].apply(lambda x: re.sub(re.compile('http\S+|https\S+'), "", x))  # REMOVE ALL URLs


# Word Embedding

#### PART A (PRETRAINED WORD2VEC)

In [11]:
word_2_vec = api.load('word2vec-google-news-300')

In [12]:
# CHECK SIMILARITY BETWEEN THE FOLLOWING 2 WORDS
word_2_vec.similarity("gorgeous", "beautiful")

0.8353004

In [13]:
# CHECK SIMILARITY BETWEEN THE FOLLOWING 2 WORDS
word_2_vec.similarity("happy", "pleased")

0.66321707

In [14]:
neighbors = word_2_vec.most_similar(positive=['man', 'woman'], negative=['boy'], topn=5)
for n in neighbors:
    print(n)

('lady', 0.5354641079902649)
('person', 0.529635488986969)
('Woman', 0.513024628162384)
('men', 0.4956325590610504)
('policewoman', 0.4909151792526245)


#### PART B (WORD2VEC TRAINED ON OUR DATA)

In [15]:
# SPLIT THE REVIEWS INTO INDIVIDUAL WORDS
sentences = []
for i in range(len(df['text'])):
    sentences.append(df['text'].values[i].split(' '))

In [37]:
model = gensim.models.Word2Vec(sentences=sentences, vector_size=300, min_count=9, window=13)

In [38]:
# CHECK SIMILARITY BETWEEN THE FOLLOWING 2 WORDS
model.wv.similarity("gorgeous", "beautiful")

0.8857682

In [39]:
# CHECK SIMILARITY BETWEEN THE FOLLOWING 2 WORDS
model.wv.similarity("happy", "pleased")

0.91375065

In [40]:
neighbors = word_2_vec.most_similar(positive=['man', 'woman'], negative=['boy'], topn=5)
for n in neighbors:
    print(n)

('lady', 0.5354641079902649)
('person', 0.529635488986969)
('Woman', 0.513024628162384)
('men', 0.4956325590610504)
('policewoman', 0.4909151792526245)


# Simple models

## Using TF-IDF Feature Extraction

In [42]:
 # Creating X data and y labels
X = df['text']
y = df['class']
    
# Applying TFIDF feature extraction on X
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=10)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(48000, 28271) (12000, 28271) (48000,) (12000,)


### Perceptron using TF-IDF

In [44]:
# FIT A SINGLE PERCEPTRON MODEL ON OUR DATASET
model1 = Perceptron(tol=1e-3, random_state=100)
model1.fit(X_train, y_train)
y_predict = model1.predict(X_test)

In [45]:
report1 = classification_report(y_predict, y_test, output_dict=True)
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           1       0.63      0.67      0.65      3749
           2       0.56      0.54      0.55      4118
           3       0.74      0.73      0.73      4133

    accuracy                           0.64     12000
   macro avg       0.64      0.65      0.64     12000
weighted avg       0.64      0.64      0.64     12000



In [46]:
print('Class\tPrecision\t\tRecall\t\t\tF1-score\n')
print('1     '+"\t"+ str(report1['1']['precision'])+",\t"+str(report1['1']['recall'])+",\t"+ str(report1['1']['f1-score']))
print('2     '+"\t"+ str(report1['2']['precision'])+",\t"+str(report1['2']['recall'])+",\t"+ str(report1['2']['f1-score']))
print('3     '+"\t"+ str(report1['3']['precision'])+",\t"+str(report1['3']['recall'])+",\t"+ str(report1['3']['f1-score']))
print('average' +"\t"+str((report1['1']['precision']+report1['2']['precision']+report1['3']['precision'])/3)+",\t"+str((report1['1']['recall']+report1['2']['recall']+report1['3']['recall'])/3)+",\t"+str((report1['1']['f1-score']+report1['2']['f1-score']+report1['3']['f1-score'])/3), '\n')

Class	Precision		Recall			F1-score

1     	0.6329050138573948,	0.6700453454254468,	0.6509458408914226
2     	0.558705469141997,	0.5407965031568722,	0.5496051332675223
3     	0.7414091470951792,	0.7256230341156545,	0.7334311567620445
average	0.6443398766981904,	0.6454882942326577,	0.6446607103069965 



### SVM using TF-IDF

In [47]:
# FIT A SVM MODEL ON OUR DATASET
model2 = LinearSVC(random_state=42)
model2.fit(X_train, y_train)
y_predict = model2.predict(X_test)

In [48]:
report2 = classification_report(y_predict, y_test, output_dict=True)
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           1       0.71      0.70      0.71      4023
           2       0.60      0.60      0.60      3975
           3       0.79      0.80      0.79      4002

    accuracy                           0.70     12000
   macro avg       0.70      0.70      0.70     12000
weighted avg       0.70      0.70      0.70     12000



In [49]:
print('Class\tPrecision\t\tRecall\t\t\tF1-score\n')
print('1     '+"\t"+ str(report2['1']['precision'])+",\t"+str(report2['1']['recall'])+",\t"+ str(report2['1']['f1-score']))
print('2     '+"\t"+ str(report2['2']['precision'])+",\t"+str(report2['2']['recall'])+",\t"+ str(report2['2']['f1-score']))
print('3     '+"\t"+ str(report2['3']['precision'])+",\t"+str(report2['3']['recall'])+",\t"+ str(report2['3']['f1-score']))
print('average' +"\t"+str((report2['1']['precision']+report2['2']['precision']+report2['3']['precision'])/3)+",\t"+str((report2['1']['recall']+report2['2']['recall']+report2['3']['recall'])/3)+",\t"+str((report2['1']['f1-score']+report2['2']['f1-score']+report2['3']['f1-score'])/3), '\n')

Class	Precision		Recall			F1-score

1     	0.7102544721592341,	0.7007208550832712,	0.7054554554554554
2     	0.5995985950827898,	0.6012578616352201,	0.6004270820248713
3     	0.7888751545117428,	0.7973513243378311,	0.7930905927674909
average	0.6995760739179223,	0.6997766803521075,	0.6996577100826059 



## Word2Vec for feature extraction

### Perceptron using Word2Vec

In [50]:
data = []
sent = []

for i in range(len(df['text'])):
    split_sent = df['text'].values[i].split(' ')
    for word in split_sent:
        try:
            sent.append(word_2_vec[word])
        except:
            sent.append(np.zeros(300))
    sent = np.array(sent)
    sent = np.mean(sent, axis=0)
    data.append(sent)
    sent = []

In [51]:
data = np.array(data)
data_label = df['class'].values
X_train1, X_test1, y_train1, y_test1 = train_test_split(data, data_label, test_size = 0.2, random_state=10)
print(X_train1.shape, X_test1.shape, y_train1.shape, y_test1.shape)

(48000, 300) (12000, 300) (48000,) (12000,)


In [52]:
# FIT A SINGLE PERCEPTRON MODEL ON OUR DATASET
model3 = Perceptron(tol=1e-3, random_state=10)
model3.fit(X_train1, y_train1)
y_predict = model3.predict(X_test1)

In [53]:
report3 = classification_report(y_predict, y_test1, output_dict=True)
print(classification_report(y_predict,y_test1))

              precision    recall  f1-score   support

           1       0.71      0.56      0.63      5019
           2       0.58      0.50      0.54      4611
           3       0.50      0.86      0.64      2370

    accuracy                           0.60     12000
   macro avg       0.60      0.64      0.60     12000
weighted avg       0.62      0.60      0.59     12000



In [54]:
print('Class\tPrecision\t\tRecall\t\t\tF1-score\n')
print('1     '+"\t"+ str(report3['1']['precision'])+",\t"+str(report3['1']['recall'])+",\t"+ str(report3['1']['f1-score']))
print('2     '+"\t"+ str(report3['2']['precision'])+",\t"+str(report3['2']['recall'])+",\t"+ str(report3['2']['f1-score']))
print('3     '+"\t"+ str(report3['3']['precision'])+",\t"+str(report3['3']['recall'])+",\t"+ str(report3['3']['f1-score']))
print('average' +"\t"+str((report3['1']['precision']+report3['2']['precision']+report3['3']['precision'])/3)+",\t"+str((report3['1']['recall']+report3['2']['recall']+report3['3']['recall'])/3)+",\t"+str((report3['1']['f1-score']+report3['2']['f1-score']+report3['3']['f1-score'])/3), '\n')

Class	Precision		Recall			F1-score

1     	0.7105064247921391,	0.5618649133293485,	0.6275033377837116
2     	0.579528349222278,	0.5009759271307742,	0.5373967663138304
3     	0.5043263288009888,	0.8607594936708861,	0.6360093530787218
average	0.5981203676051353,	0.6412001113770031,	0.600303152392088 



### SVM using Word2Vec

In [55]:
# FIT A SVM MODEL ON OUR DATASET
model4 = LinearSVC(random_state=100)
model4.fit(X_train1, y_train1)
y_predict = model4.predict(X_test1)

In [56]:
report4 = classification_report(y_predict, y_test1, output_dict=True)
print(classification_report(y_predict,y_test1))

              precision    recall  f1-score   support

           1       0.69      0.67      0.68      4094
           2       0.57      0.58      0.57      3920
           3       0.75      0.76      0.75      3986

    accuracy                           0.67     12000
   macro avg       0.67      0.67      0.67     12000
weighted avg       0.67      0.67      0.67     12000



In [57]:
print('Class\tPrecision\t\tRecall\t\t\tF1-score\n')
print('1     '+"\t"+ str(report4['1']['precision'])+",\t"+str(report4['1']['recall'])+",\t"+ str(report4['1']['f1-score']))
print('2     '+"\t"+ str(report4['2']['precision'])+",\t"+str(report4['2']['recall'])+",\t"+ str(report4['2']['f1-score']))
print('3     '+"\t"+ str(report4['3']['precision'])+",\t"+str(report4['3']['recall'])+",\t"+ str(report4['3']['f1-score']))
print('average' +"\t"+str((report4['1']['precision']+report4['2']['precision']+report4['3']['precision'])/3)+",\t"+str((report4['1']['recall']+report4['2']['recall']+report4['3']['recall'])/3)+",\t"+str((report4['1']['f1-score']+report4['2']['f1-score']+report4['3']['f1-score'])/3), '\n')

Class	Precision		Recall			F1-score

1     	0.690854119425548,	0.6697606253053249,	0.6801438670470048
2     	0.5669844455594582,	0.576530612244898,	0.5717176827725778
3     	0.7456118665018542,	0.7566482689412946,	0.751089528078695
average	0.6678168104956201,	0.6676465021638393,	0.6676503592994258 



In [59]:
pred = model1.predict(X_test)
acc = accuracy_score(y_test, pred) * 100
pred1 = model3.predict(X_test1)
acc1 = accuracy_score(y_test1, pred1) * 100
print("FOR PERCEPTRON -")
print("Test accuracy when using TF-IDF:", acc, "%")
print("Test accuracy when using Word2Vec:", acc1, "%")

FOR PERCEPTRON -
Test accuracy when using TF-IDF: 64.48333333333333 %
Test accuracy when using Word2Vec: 59.75 %


In [60]:
pred3 = model2.predict(X_test)
acc = accuracy_score(y_test, pred3) * 100
pred4 = model4.predict(X_test1)
acc1 = accuracy_score(y_test1, pred4) * 100
print("FOR SVM -")
print("Test accuracy when using TF-IDF:", acc, "%")
print("Test accuracy when using Word2Vec:", acc1, '%')

FOR SVM -
Test accuracy when using TF-IDF: 70.0 %
Test accuracy when using Word2Vec: 66.81666666666666 %


##### TF-IDF gives better results than Word2Vec for both Perceptron and SVM

# Feedforward Neural Networks

#### PART A

In [61]:
# CREATE ONE-HOT ENCODINGS OF THE 3 RATINGS
enc_data_label = np.zeros((len(data_label), 3))
for i in range(len(data_label)):
    if data_label[i] == 1:
        enc_data_label[i] = [0, 0, 1]
    elif data_label[i] == 2:
        enc_data_label[i] = [0, 1, 0]
    elif data_label[i] == 3:
        enc_data_label[i] = [1, 0, 0]

In [62]:
# TRAIN-TEST-SPLIT
X_train, X_test, y_train, y_test = train_test_split(data, enc_data_label, test_size = 0.2, random_state=10)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(48000, 300) (12000, 300) (48000, 3) (12000, 3)


### Other activation functions used:
- https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html#torch.nn.LeakyReLU
- https://pytorch.org/docs/stable/generated/torch.nn.Tanh.html#torch.nn.Tanh

In [64]:
# FNN IN PYTORCH
class FNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(300, 100)   
        self.dropout = nn.Dropout(0.2)   
        self.fc2 = nn.Linear(100, 10)
        self.fc3 = nn.Linear(10, 3)
        
    def forward(self,x):
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)     # added dropout to reduce overfitting 
        x = nn.functional.relu(self.fc2(x))
#         x = self.dropout(x)   # added dropout to reduce overfitting 
        x = nn.functional.relu(self.fc3(x))
        
        return x

In [65]:
# USING GPU
print(torch.cuda.device_count())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

2


device(type='cuda', index=0)

In [66]:
fnn_model = FNN().to(device)
fnn_model = fnn_model.to(device)   # using '.to(device)' to move the model from CPU to GPU

In [67]:
# CREATE DATASET CLASS FOR DATALOADERS
class Dataset(Dataset):
    def __init__(self, data1, data2):
        self.data1 = data1
        self.data2 = data2
        
    def __len__(self):
        return len(self.data1)
    
    def __getitem__(self, idx):
        X = self.data1[idx]
        y = self.data2[idx]

        return X, y

In [69]:
batch_size = 512  # BATCH SIZE FOR THIS MODEL
train_dataset = Dataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

test_dataset = Dataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

In [70]:
model_optim=torch.optim.Adam(fnn_model.parameters(), lr = 0.001)  
CEloss = nn.CrossEntropyLoss()

In [71]:
# TRAINING THE FNN MODEL
history_train = []
history_test = []
train_dataloader_len = len(train_dataloader)
test_dataloader_len = len(test_dataloader)
train_len = X_train.shape[0]
test_len = X_test.shape[0]

for epoch in range(30):  # loops over the complete dataset multiple times (which is the nummber of epochs)
    fnn_model.train()     
    train_loss = 0.0
    correct_train = 0
    train_accuracy = 0.0
    for i, data in enumerate(train_dataloader, 0):  # loops over complete training dataset once 
        
        inputs, label = data
        inputs = inputs.float()    # convert double values to float
        inputs, label = inputs.to(device), label.to(device)   

        model_optim.zero_grad()
        output = fnn_model(inputs)   # forward pass of model
        output = output.to(device)
        
        loss1 = CEloss(output, label)     # loss calculation
        loss1.backward()            # computes the gradient during the backward pass
        model_optim.step()   # performs single optimization step

        train_loss += loss1.item()   # adding accuracy values of all batches in an epoch
        _, output = torch.max(output, 1)     # storing the index of maximum value in prediction to the variable 'output'
        output = output.cpu().detach().numpy()     # loads the variable to cpu and converts it to a numpy array
        label = label.cpu().detach().numpy()        
        label = np.argmax(label, axis = 1)   # storing the index of maximum value in label to the variable 'label'
        train_accuracy += accuracy_score(label, output)  # adding accuracy values of all batches in training dataset in an epoch
    
    train_loss = train_loss/train_dataloader_len
    train_accuracy = train_accuracy/train_dataloader_len  # dividing accuracy by number of batches for training dataset
    history_train.append((train_loss, train_accuracy))
    
    fnn_model.eval()     # model evaluation on test dataset
    test_loss = 0.0
    test_accuracy = 0.0
    correct_test = 0
    with torch.no_grad():     # disables gradient calculation
        for i, data in enumerate(test_dataloader, 0):    # loops over complete test dataset once
            
            inputs, label = data
            inputs = inputs.float()
            inputs, label = inputs.to(device), label.to(device)

            pred = fnn_model(inputs)
            pred = pred.to(device)
            loss2 = CEloss(pred, label)

            test_loss += loss2.item()
            _, pred = torch.max(pred, 1)
            pred = pred.cpu().detach().numpy()
            label = label.cpu().detach().numpy()
            label = np.argmax(label, axis = 1)    
            test_accuracy += accuracy_score(label, pred)
            
        test_loss = test_loss/test_dataloader_len
        test_accuracy = test_accuracy/test_dataloader_len
        history_test.append((test_loss, test_accuracy))
    
#     print("Epoch:", (epoch+1))
#     print("Train Loss:", train_loss, "\tTrain Accuracy:", train_accuracy)
#     print("Test Loss:", test_loss, "\tTest Accuracy:", test_accuracy)
#     print("=======================================================================================")
    print("Train accuracy:", train_accuracy)

print("Final test accuracy:", test_accuracy)

Train accuracy: 0.41718611480496454
Train accuracy: 0.5781111480496454
Train accuracy: 0.6403341090425532
Train accuracy: 0.6577598625886525
Train accuracy: 0.6649767287234043
Train accuracy: 0.668287344858156
Train accuracy: 0.6708984375
Train accuracy: 0.6747977615248227
Train accuracy: 0.6778105607269503
Train accuracy: 0.6818622562056738
Train accuracy: 0.6817583665780141
Train accuracy: 0.6856438386524822
Train accuracy: 0.6881441156914894
Train accuracy: 0.6896886081560284
Train accuracy: 0.6906720966312057
Train accuracy: 0.6909145057624114
Train accuracy: 0.6928745567375886
Train accuracy: 0.6950216090425532
Train accuracy: 0.6962198027482269
Train accuracy: 0.6969608820921986
Train accuracy: 0.6979097406914894
Train accuracy: 0.6974734042553191
Train accuracy: 0.6995442708333333
Train accuracy: 0.7021830673758864
Train accuracy: 0.7024462544326242
Train accuracy: 0.7029657025709221
Train accuracy: 0.7037829676418439
Train accuracy: 0.705576795212766
Train accuracy: 0.705902316

#### PART B

In [80]:
# concatenate the first 10 Word2Vec vectors for each review
concat_data = []
sent = []
for i in range(len(df['text'])):
    split_sent = df['text'].values[i].split(' ')
    for word in split_sent[:10]:
        try:
            sent.append(word_2_vec[word])
        except:
            
            sent.append(list(np.zeros(300)))   
    if len(sent) < 10:
        sent = np.concatenate([sent,np.zeros((10-len(sent), 300))])
    sent = np.array(sent)
    sent = sent.flatten()
    concat_data.append(sent)
    sent = []
concat_data = np.array(concat_data)

In [81]:
class FNN1(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(3000, 100)
        self.dropout = nn.Dropout(0.2)   
        self.fc2 = nn.Linear(100, 10)
        self.fc3 = nn.Linear(10, 3)
        
    def forward(self,x):
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)     # added dropout to reduce overfitting 
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        
        return x

In [88]:
fnn_model1 = FNN1().to(device)
fnn_model1 = fnn_model1.to(device)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(concat_data, enc_data_label, test_size = 0.2, random_state=10)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(48000, 3000) (12000, 3000) (48000, 3) (12000, 3)


In [90]:
batch_size = 512
train_dataset = Dataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

test_dataset = Dataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

In [91]:
model_optim=torch.optim.Adam(fnn_model1.parameters(), lr = 0.001)
CEloss = nn.CrossEntropyLoss()

In [92]:
history_train = []
history_test = []
train_dataloader_len = len(train_dataloader)
test_dataloader_len = len(test_dataloader)
train_len = X_train.shape[0]
test_len = X_test.shape[0]

for epoch in range(10):  # loops over only 15 epochs because the model starts overfitting heavily after that
    fnn_model1.train()
    train_loss = 0.0
    correct_train = 0
    train_accuracy = 0.0
    for i, data in enumerate(train_dataloader, 0):
        
        inputs, label = data
        inputs = inputs.float()
        inputs, label = inputs.to(device), label.to(device)

        model_optim.zero_grad()
        output = fnn_model1(inputs)
        output = output.to(device)
        loss1 = CEloss(output, label)
        loss1.backward()
        model_optim.step()

        train_loss += loss1.item()
        _, output = torch.max(output, 1)
        output = output.cpu().detach().numpy()
        label = label.cpu().detach().numpy()
        label = np.argmax(label, axis = 1)
        train_accuracy += accuracy_score(label, output)
    
    train_loss = train_loss/train_dataloader_len
    train_accuracy = train_accuracy/train_dataloader_len
    history_train.append((train_loss, train_accuracy))
    
    fnn_model1.eval()   
    test_loss = 0.0
    test_accuracy = 0.0
    correct_test = 0
    with torch.no_grad():
        for i, data in enumerate(test_dataloader, 0):
            
            inputs, label = data
            inputs = inputs.float()
            inputs, label = inputs.to(device), label.to(device)

            pred = fnn_model1(inputs)
            pred = pred.to(device)
            loss2 = CEloss(pred, label)

            test_loss += loss2.item()
            _, pred = torch.max(pred, 1)
            pred = pred.cpu().detach().numpy()
            label = label.cpu().detach().numpy()
            label = np.argmax(label, axis = 1)
            test_accuracy += accuracy_score(label, pred)
            
        test_loss = test_loss/test_dataloader_len
        test_accuracy = test_accuracy/test_dataloader_len
        history_test.append((test_loss, test_accuracy))
    
#     print("Epoch:", (epoch+1))
#     print("Train Loss:", train_loss, "\tTrain Accuracy:", train_accuracy)
#     print("Test Loss:", test_loss, "\tTest Accuracy:", test_accuracy)
#     print("=======================================================================================")
    print("Train accuracy:", train_accuracy)

print("Final test accuracy:", test_accuracy)

Train accuracy: 0.546126994680851
Train accuracy: 0.6177138741134751
Train accuracy: 0.6393921764184397
Train accuracy: 0.6540613918439716
Train accuracy: 0.6709192154255319
Train accuracy: 0.6910322473404256
Train accuracy: 0.7137979277482269
Train accuracy: 0.7340009973404256
Train accuracy: 0.7574384973404256
Train accuracy: 0.7788605385638298
Final test accuracy: 0.6060965401785714


# Recurrent Neural Networks

#### PART A (Simple RNN)

In [93]:
# Limiting the maximum review length to 20 by truncating longer reviews and padding shorter reviews with a null value 0 (for creating our dataset)
new_data = []
sent = []

for i in range(len(df['text'])):
    split_sent = df['text'].values[i].split(' ')
    for word in split_sent[:20]:
        sent.append(word)

    if len(split_sent) < 20:
        for i in range(20-len(split_sent)):
            sent.append('0')
    
    new_data.append(sent)
    sent = []

In [94]:
rnn_data = []
sent = []

for i in range(len(new_data)):
    sent_part = new_data[i]
    for word in sent_part:
        try:
            sent.append(word_2_vec[word])
        except:
            sent.append(np.zeros(300))
    sent = np.array(sent)
    rnn_data.append(sent)
    sent = []
rnn_data = np.array(rnn_data)

In [95]:
X_train, X_test, y_train, y_test = train_test_split(rnn_data, enc_data_label, test_size = 0.2, random_state=10)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(48000, 20, 300) (12000, 20, 300) (48000, 3) (12000, 3)


In [96]:
batch_size = 512
train_dataset = Dataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

test_dataset = Dataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

In [97]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rnn = nn.RNN(300, 20, 1, batch_first = True, nonlinearity='relu')
        self.fc1 = nn.Linear(20, 3)
        
    def forward(self,x):
         
        x, hn = self.rnn(x)
        x = nn.functional.relu(self.fc1(x[:, -1, :]))
        
        return x

In [101]:
rnn_model = RNN().to(device)
rnn_model = rnn_model.to(device)

In [102]:
model_optim=torch.optim.Adam(rnn_model.parameters(), lr = 0.001)
CEloss = nn.CrossEntropyLoss()

In [103]:
history_train = []
history_test = []
train_dataloader_len = len(train_dataloader)
test_dataloader_len = len(test_dataloader)
train_len = X_train.shape[0]
test_len = X_test.shape[0]

for epoch in range(40):  
    rnn_model.train()
    train_loss = 0.0
    correct_train = 0
    train_accuracy = 0.0
    for i, data in enumerate(train_dataloader, 0):
        
        inputs, label = data
        inputs = inputs.float()
        inputs, label = inputs.to(device), label.to(device)

        model_optim.zero_grad()
        output = rnn_model(inputs)
        output = output.to(device)
        loss1 = CEloss(output, label)
        loss1.backward()
        model_optim.step()

        train_loss += loss1.item()
        _, output = torch.max(output, 1)
        output = output.cpu().detach().numpy()
        label = label.cpu().detach().numpy()
        label = np.argmax(label, axis = 1)
        train_accuracy += accuracy_score(label, output)
    
    train_loss = train_loss/train_dataloader_len
    train_accuracy = train_accuracy/train_dataloader_len
    history_train.append((train_loss, train_accuracy))
    
    rnn_model.eval()   
    test_loss = 0.0
    test_accuracy = 0.0
    correct_test = 0
    with torch.no_grad():
        for i, data in enumerate(test_dataloader, 0):
            
            inputs, label = data
            inputs = inputs.float()
            inputs, label = inputs.to(device), label.to(device)

            pred = rnn_model(inputs)
            pred = pred.to(device)
            loss2 = CEloss(pred, label)

            test_loss += loss2.item()
            _, pred = torch.max(pred, 1)
            pred = pred.cpu().detach().numpy()
            label = label.cpu().detach().numpy()
            label = np.argmax(label, axis = 1)
            test_accuracy += accuracy_score(label, pred)
            
        test_loss = test_loss/test_dataloader_len
        test_accuracy = test_accuracy/test_dataloader_len
        history_test.append((test_loss, test_accuracy))
    
#     print("Epoch:", (epoch+1))
#     print("Train Loss:", train_loss, "\tTrain Accuracy:", train_accuracy)
#     print("Test Loss:", test_loss, "\tTest Accuracy:", test_accuracy)
#     print("=======================================================================================")
    print("Train accuracy:", train_accuracy)

print("Final test accuracy:", test_accuracy)

Train accuracy: 0.36236702127659576
Train accuracy: 0.40335494237588654
Train accuracy: 0.5445617242907802
Train accuracy: 0.5819758421985816
Train accuracy: 0.5986743683510638
Train accuracy: 0.6095135195035462
Train accuracy: 0.6140431072695036
Train accuracy: 0.6197154809397163
Train accuracy: 0.6250692597517731
Train accuracy: 0.6271678302304964
Train accuracy: 0.6314619348404256
Train accuracy: 0.6339206560283688
Train accuracy: 0.6362062278368794
Train accuracy: 0.6364624889184397
Train accuracy: 0.6418162677304964
Train accuracy: 0.6444966201241135
Train accuracy: 0.6458887411347517
Train accuracy: 0.6455216644503546
Train accuracy: 0.6476617907801419
Train accuracy: 0.6482989804964538
Train accuracy: 0.6517619680851063
Train accuracy: 0.6535211657801419
Train accuracy: 0.6512840757978723
Train accuracy: 0.6547816932624113
Train accuracy: 0.6589857601950355
Train accuracy: 0.6588403147163121
Train accuracy: 0.6606202903368794
Train accuracy: 0.6619708554964538
Train accuracy: 0.

##### FNN gives better test accuracy than Simple RNN. But the dataset (input) is different for Simple RNN and FNN. If the dataset would have been same, RNN would have given better test accuracy.

#### PART B (GRU)

In [104]:
class GRU(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.gru = nn.GRU(300, 20, 1, batch_first = True)
        self.fc1 = nn.Linear(20, 3)
        
    def forward(self,x):
         
        x, hn = self.gru(x)
        x = nn.functional.relu(self.fc1(x[:, -1, :]))
        
        return x

In [105]:
gru_model = GRU().to(device)
gru_model = gru_model.to(device)

In [106]:
model_optim=torch.optim.Adam(gru_model.parameters(), lr = 0.001)
CEloss = nn.CrossEntropyLoss()

In [107]:
history_train = []
history_test = []
train_dataloader_len = len(train_dataloader)
test_dataloader_len = len(test_dataloader)
train_len = X_train.shape[0]
test_len = X_test.shape[0]

for epoch in range(40):  
    gru_model.train()
    train_loss = 0.0
    correct_train = 0
    train_accuracy = 0.0
    for i, data in enumerate(train_dataloader, 0):
        
        inputs, label = data
        inputs = inputs.float()
        inputs, label = inputs.to(device), label.to(device)

        model_optim.zero_grad()
        output = gru_model(inputs)
        output = output.to(device)
        loss1 = CEloss(output, label)
        loss1.backward()
        model_optim.step()

        train_loss += loss1.item()
        _, output = torch.max(output, 1)
        output = output.cpu().detach().numpy()
        label = label.cpu().detach().numpy()
        label = np.argmax(label, axis = 1)
        train_accuracy += accuracy_score(label, output)
    
    train_loss = train_loss/train_dataloader_len
    train_accuracy = train_accuracy/train_dataloader_len
    history_train.append((train_loss, train_accuracy))
    
    gru_model.eval()   
    test_loss = 0.0
    test_accuracy = 0.0
    correct_test = 0
    with torch.no_grad():
        for i, data in enumerate(test_dataloader, 0):
            
            inputs, label = data
            inputs = inputs.float()
            inputs, label = inputs.to(device), label.to(device)

            pred = gru_model(inputs)
            pred = pred.to(device)
            loss2 = CEloss(pred, label)

            test_loss += loss2.item()
            _, pred = torch.max(pred, 1)
            pred = pred.cpu().detach().numpy()
            label = label.cpu().detach().numpy()
            label = np.argmax(label, axis = 1)
            test_accuracy += accuracy_score(label, pred)
            
        test_loss = test_loss/test_dataloader_len
        test_accuracy = test_accuracy/test_dataloader_len
        history_test.append((test_loss, test_accuracy))
    
#     print("Epoch:", (epoch+1))
#     print("Train Loss:", train_loss, "\tTrain Accuracy:", train_accuracy)
#     print("Test Loss:", test_loss, "\tTest Accuracy:", test_accuracy)
#     print("=======================================================================================")
    print("Train accuracy:", train_accuracy)
print("Final test accuracy:", test_accuracy)

Train accuracy: 0.37563026374113473
Train accuracy: 0.5224678634751773
Train accuracy: 0.6130388408687943
Train accuracy: 0.6416777482269503
Train accuracy: 0.658521719858156
Train accuracy: 0.6698526152482269
Train accuracy: 0.6727199689716312
Train accuracy: 0.6785931959219857
Train accuracy: 0.6806155806737589
Train accuracy: 0.6861979166666667
Train accuracy: 0.6874307402482269
Train accuracy: 0.6905959109042553
Train accuracy: 0.6939827127659575
Train accuracy: 0.6948276817375886
Train accuracy: 0.6992741578014184
Train accuracy: 0.6974734042553191
Train accuracy: 0.7002091644503545
Train accuracy: 0.7024670323581561
Train accuracy: 0.7037552637411348
Train accuracy: 0.7050227171985816
Train accuracy: 0.707121287677305
Train accuracy: 0.7081186281028369
Train accuracy: 0.7086588541666667
Train accuracy: 0.7124404366134752
Train accuracy: 0.7141580784574468
Train accuracy: 0.7138187056737588
Train accuracy: 0.7138533355496455
Train accuracy: 0.7167760970744681
Train accuracy: 0.717

#### PART C (LSTM)

In [108]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lstm = nn.LSTM(300, 20, 1, batch_first = True)
        self.fc1 = nn.Linear(20, 3)
        
    def forward(self,x):
         
        x, (hn, cn) = self.lstm(x)
        x = nn.functional.relu(self.fc1(x[:, -1, :]))
        
        return x

In [109]:
lstm_model = LSTM().to(device)
lstm_model = lstm_model.to(device)

In [110]:
model_optim=torch.optim.Adam(lstm_model.parameters(), lr = 0.001)
CEloss = nn.CrossEntropyLoss()

In [111]:
history_train = []
history_test = []
train_dataloader_len = len(train_dataloader)
test_dataloader_len = len(test_dataloader)
train_len = X_train.shape[0]
test_len = X_test.shape[0]

for epoch in range(40): 
    lstm_model.train()
    train_loss = 0.0
    correct_train = 0
    train_accuracy = 0.0
    for i, data in enumerate(train_dataloader, 0):
        
        inputs, label = data
        inputs = inputs.float()
        inputs, label = inputs.to(device), label.to(device)

        model_optim.zero_grad()
        output = lstm_model(inputs)
        output = output.to(device)
        loss1 = CEloss(output, label)
        loss1.backward()
        model_optim.step()

        train_loss += loss1.item()
        _, output = torch.max(output, 1)
        output = output.cpu().detach().numpy()
        label = label.cpu().detach().numpy()
        label = np.argmax(label, axis = 1)
        train_accuracy += accuracy_score(label, output)
    
    train_loss = train_loss/train_dataloader_len
    train_accuracy = train_accuracy/train_dataloader_len
    history_train.append((train_loss, train_accuracy))
    
    lstm_model.eval()   
    test_loss = 0.0
    test_accuracy = 0.0
    correct_test = 0
    with torch.no_grad():
        for i, data in enumerate(test_dataloader, 0):
            
            inputs, label = data
            inputs = inputs.float()
            inputs, label = inputs.to(device), label.to(device)

            pred = lstm_model(inputs)
            pred = pred.to(device)
            loss2 = CEloss(pred, label)

            test_loss += loss2.item()
            _, pred = torch.max(pred, 1)
            pred = pred.cpu().detach().numpy()
            label = label.cpu().detach().numpy()
            label = np.argmax(label, axis = 1)
            test_accuracy += accuracy_score(label, pred)
            
        test_loss = test_loss/test_dataloader_len
        test_accuracy = test_accuracy/test_dataloader_len
        history_test.append((test_loss, test_accuracy))
    
#     print("Epoch:", (epoch+1))
#     print("Train Loss:", train_loss, "\tTrain Accuracy:", train_accuracy)
#     print("Test Loss:", test_loss, "\tTest Accuracy:", test_accuracy)
#     print("=======================================================================================")
    print("Train accuracy:", train_accuracy)

print("Final test accuracy:", test_accuracy)

Train accuracy: 0.38286790780141844
Train accuracy: 0.509537067819149
Train accuracy: 0.609942929964539
Train accuracy: 0.6386926529255319
Train accuracy: 0.6527039007092198
Train accuracy: 0.659747617464539
Train accuracy: 0.6685436059397163
Train accuracy: 0.6707876218971631
Train accuracy: 0.6771179632092198
Train accuracy: 0.67853086214539
Train accuracy: 0.6827695589539008
Train accuracy: 0.6838361591312057
Train accuracy: 0.6894877548758864
Train accuracy: 0.689591644503546
Train accuracy: 0.6921057734929078
Train accuracy: 0.6966007313829787
Train accuracy: 0.6983530031028369
Train accuracy: 0.6996343085106383
Train accuracy: 0.7026817375886524
Train accuracy: 0.7042539339539008
Train accuracy: 0.7065949135638298
Train accuracy: 0.7080562943262412
Train accuracy: 0.7098362699468085
Train accuracy: 0.7099193816489362
Train accuracy: 0.7113322805851063
Train accuracy: 0.7145528590425532
Train accuracy: 0.7148021941489362
Train accuracy: 0.7178149933510638
Train accuracy: 0.7155155

##### We can conclude that GRU and LSTM give better results than Simple RNN for our dataset

Reference: https://github.com/27rg5/Sentiment-Analysis-using-Deep-Learning/tree/master